In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Kidney_Clear_Cell_Carcinoma"
cohort = "GSE119958"

# Input paths
in_trait_dir = "../../input/GEO/Kidney_Clear_Cell_Carcinoma"
in_cohort_dir = "../../input/GEO/Kidney_Clear_Cell_Carcinoma/GSE119958"

# Output paths
out_data_file = "../../output/preprocess/Kidney_Clear_Cell_Carcinoma/GSE119958.csv"
out_gene_data_file = "../../output/preprocess/Kidney_Clear_Cell_Carcinoma/gene_data/GSE119958.csv"
out_clinical_data_file = "../../output/preprocess/Kidney_Clear_Cell_Carcinoma/clinical_data/GSE119958.csv"
json_path = "../../output/preprocess/Kidney_Clear_Cell_Carcinoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"APOL1 renal-risk variants induce mitochondrial fission"
!Series_summary	"To assess gene expression by APOL1 genotypes in primary proximal tubule cells (PTCs), global gene expression (mRNA) levels were examined on Affymetrix HTA 2.0 arrays in primary PTCs cultured from non-diseased kidney in African Americans without CKD who underwent nephrectomy for localized renal cell carcinoma."
!Series_overall_design	"To detect gene profiles attributable to APOL1 renal-risk genotypes, African American primary proximal tubule cells with two APOL1 renal-risk alleles (N=5), one risk allele (N=23),  and lacking renal-risk alleles (N=22) were compared for  global gene expression when poly IC stimulation was absent (pre-) or present (post-) ."
Sample Characteristics Dictionary:
{0: ['ethnicity: African American'], 1: ['genotype: APOL1 Genotype G0G0', 'genotype: APOL1 Genotype G1G2', 'genotype: APOL1 Genotype G2G0', 'genotype: APOL1 Genotype G1G0', 'genotype: APOL1 G

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# From the background info, we can see that this dataset contains gene expression data from Affymetrix arrays
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Kidney Clear Cell Carcinoma):
# The data is from African Americans without CKD who underwent nephrectomy for localized renal cell carcinoma
# This suggests we have data about renal cell carcinoma patients
trait_row = None  # Not directly available in the sample characteristics

# For age:
age_row = None  # Not available in the sample characteristics

# For gender:
gender_row = None  # Not available in the sample characteristics

# 2.2 Data Type Conversion

# For trait: 
# Since we don't have a direct trait row, we'll define a function for completeness
def convert_trait(value):
    return None

# For age:
def convert_age(value):
    return None

# For gender:
def convert_gender(value):
    return None

# 3. Save Metadata
# Trait data is not directly available in the sample characteristics
is_trait_available = False

# Validate and save cohort information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we'll skip this step


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Add diagnostic code to check file content and structure
print("Examining matrix file structure...")
with gzip.open(matrix_file, 'rt') as file:
    table_marker_found = False
    lines_read = 0
    for i, line in enumerate(file):
        lines_read += 1
        if '!series_matrix_table_begin' in line:
            table_marker_found = True
            print(f"Found table marker at line {i}")
            # Read a few lines after the marker to check data structure
            next_lines = [next(file, "").strip() for _ in range(5)]
            print("First few lines after marker:")
            for next_line in next_lines:
                print(next_line)
            break
        if i < 10:  # Print first few lines to see file structure
            print(f"Line {i}: {line.strip()}")
        if i > 100:  # Don't read the entire file
            break
    
    if not table_marker_found:
        print("Table marker '!series_matrix_table_begin' not found in first 100 lines")
    print(f"Total lines examined: {lines_read}")

# 2. Try extracting gene expression data from the matrix file again with better diagnostics
try:
    print("\nAttempting to extract gene data from matrix file...")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {str(e)}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")

# If data extraction failed, try an alternative approach using pandas directly
if not is_gene_available:
    print("\nTrying alternative approach to read gene expression data...")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            # Skip lines until we find the marker
            for line in file:
                if '!series_matrix_table_begin' in line:
                    break
            
            # Try to read the data directly with pandas
            gene_data = pd.read_csv(file, sep='\t', index_col=0)
            
            if not gene_data.empty:
                print(f"Successfully extracted gene data with alternative method: {gene_data.shape}")
                print("First 20 gene IDs:")
                print(gene_data.index[:20])
                is_gene_available = True
            else:
                print("Alternative extraction method also produced empty data")
    except Exception as e:
        print(f"Alternative extraction failed: {str(e)}")


Examining matrix file structure...
Line 0: !Series_title	"APOL1 renal-risk variants induce mitochondrial fission"
Line 1: !Series_geo_accession	"GSE119958"
Line 2: !Series_status	"Public on Sep 15 2018"
Line 3: !Series_submission_date	"Sep 14 2018"
Line 4: !Series_last_update_date	"Apr 04 2020"
Line 5: !Series_summary	"To assess gene expression by APOL1 genotypes in primary proximal tubule cells (PTCs), global gene expression (mRNA) levels were examined on Affymetrix HTA 2.0 arrays in primary PTCs cultured from non-diseased kidney in African Americans without CKD who underwent nephrectomy for localized renal cell carcinoma."
Line 6: !Series_overall_design	"To detect gene profiles attributable to APOL1 renal-risk genotypes, African American primary proximal tubule cells with two APOL1 renal-risk alleles (N=5), one risk allele (N=23),  and lacking renal-risk alleles (N=22) were compared for  global gene expression when poly IC stimulation was absent (pre-) or present (post-) ."
Line 7: !

Successfully extracted gene data with 67528 rows
First 20 gene IDs:
Index(['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1',
       'TC01000004.hg.1', 'TC01000005.hg.1', 'TC01000006.hg.1',
       'TC01000007.hg.1', 'TC01000008.hg.1', 'TC01000009.hg.1',
       'TC01000010.hg.1', 'TC01000011.hg.1', 'TC01000012.hg.1',
       'TC01000013.hg.1', 'TC01000014.hg.1', 'TC01000015.hg.1',
       'TC01000016.hg.1', 'TC01000017.hg.1', 'TC01000018.hg.1',
       'TC01000019.hg.1', 'TC01000020.hg.1'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# Based on the gene identifiers like "TC01000001.hg.1", I can determine these are not
# standard human gene symbols but rather Affymetrix probe identifiers from the HTA 2.0 arrays.
# This is confirmed by the series summary mentioning "Affymetrix HTA 2.0 arrays".
# These probe IDs need to be mapped to human gene symbols for biological interpretation.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 6823653 rows

Gene annotation preview (first few rows):
{'ID': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'probeset_id': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['11869', '29554', '69091', '160446', '317811'], 'stop': ['14409', '31109', '70008', '161525', '328581'], 'total_probes': [49.0, 60.0, 30.0, 30.0, 191.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// ENST00000456328 // DDX11L5 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 5 // 9p24.3 // 100287596 /// ENST00000456328 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102', 'ENST00000408384 // MIR1302-11 // microRNA 1302-11 // --- // 100422919 /// ENST0000

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Decide which columns in the gene annotation to use for mapping
# From examining the data, I can see:
# - 'ID' in gene_annotation matches the index of gene_data (TC01000001.hg.1, etc.)
# - 'gene_assignment' contains gene symbols embedded within longer strings

print("Processing gene mapping...")
# Extract mapping between probe IDs and gene symbols
prob_col = 'ID'
gene_col = 'gene_assignment'

# 2. Get the gene mapping dataframe
mapping_data = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"Created mapping data with {len(mapping_data)} rows")

# Display sample of mapping to verify
print("\nFirst few rows of mapping data:")
print(preview_df(mapping_data.head()))

# 3. Convert probe-level measurements to gene expression data
print("\nApplying gene mapping to convert probe measurements to gene expression data...")
gene_data = apply_gene_mapping(gene_data, mapping_data)
print(f"Generated gene expression data with {len(gene_data)} genes and {len(gene_data.columns)} samples")

# Display a sample of the gene expression data
print("\nFirst few rows of gene expression data:")
print(preview_df(gene_data.head()))

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Saved gene expression data to {out_gene_data_file}")


Processing gene mapping...


Created mapping data with 70753 rows

First few rows of mapping data:
{'ID': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'Gene': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// ENST00000456328 // DDX11L5 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 5 // 9p24.3 // 100287596 /// ENST00000456328 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102', 'ENST00000408384 // MIR1302-11 // microRNA 1302-11 // --- // 100422919 /// ENST00000408384 // MIR1302-10 // microRNA 1302-10 // --- // 100422834 /// ENST00000408384 // MIR1302-9 // microRNA 1302-9 // --- // 100422831 /// ENST00000408384 // MIR1302-2 // microRNA 1302-2 // --- // 100302278 /// ENST00000469289 // MIR1302-11 // microRNA 1302-11 // --- // 100422919 /// ENST00000469289 // MIR1302-10 // microRNA 1302-10 // --- // 100422834 /// ENST00000469289 // MIR1302-9 // microRNA 1302-9 // ---

Generated gene expression data with 71527 genes and 100 samples

First few rows of gene expression data:
{'GSM3389407': [18.932030916666665, 1.1775772, 5.371056666666667, 1.74514775, 1.5487551666666668], 'GSM3389408': [19.046464791666665, 1.1226058, 5.256726666666666, 1.71101625, 1.6268235], 'GSM3389409': [19.016802333333334, 1.1384794, 5.38398, 1.70044775, 1.5824381666666665], 'GSM3389410': [19.459400166666665, 1.1460666, 5.2680066666666665, 1.70947075, 1.5796898333333331], 'GSM3389411': [19.337880416666664, 1.114715, 5.25694, 1.70998025, 1.6002086666666666], 'GSM3389412': [19.337805958333334, 1.1405474, 5.269506666666667, 1.7845565, 1.5282928333333332], 'GSM3389413': [19.041272333333332, 1.1281822, 5.144073333333333, 1.76992025, 1.6196983333333335], 'GSM3389414': [19.33273375, 1.1147008, 5.259906666666667, 1.7724995, 1.702872], 'GSM3389415': [19.554414958333332, 1.1131541999999999, 5.1313466666666665, 1.73526125, 1.6858913333333334], 'GSM3389416': [19.23352925, 1.112511, 5.25888, 1.7

Saved gene expression data to ../../output/preprocess/Kidney_Clear_Cell_Carcinoma/gene_data/GSE119958.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("\nNormalizing gene symbols...")
try:
    # Check if gene_data is empty before normalization
    if gene_data.empty:
        print("Gene data is empty. Skipping normalization.")
        normalized_gene_data = gene_data
    else:
        normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
        print(f"After normalization: {len(normalized_gene_data.index)} unique gene symbols")
    
    # Save the normalized gene data (even if empty, to maintain consistent workflow)
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene expression data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error normalizing gene symbols: {e}")
    normalized_gene_data = gene_data  # Use original data if normalization fails

# Use is_trait_available from previous steps, don't recheck trait_row
print("\nChecking trait availability from previous analysis...")
# Load the clinical data from the previously saved file
try:
    clinical_df = pd.read_csv(out_clinical_data_file)
    print(f"Loaded clinical data with shape: {clinical_df.shape}")
    # Rename the column '1' to the trait name if it exists
    if '1' in clinical_df.columns:
        clinical_df = clinical_df.rename(columns={'1': trait})
        print(f"Renamed column '1' to '{trait}'")
    is_trait_available = trait in clinical_df.columns
except Exception as e:
    print(f"Error loading clinical data: {e}")
    clinical_df = pd.DataFrame()
    is_trait_available = False

# 2. Link clinical and genetic data if available
print("\nLinking clinical and genetic data...")
try:
    if is_trait_available and not normalized_gene_data.empty:
        # Print sample IDs from both datasets for debugging
        print("Clinical columns:", list(clinical_df.columns))
        print("First few genetic sample columns:", list(normalized_gene_data.columns)[:5])
        
        # Transpose clinical data so samples are rows
        if clinical_df.shape[0] == 1:  # If it's currently 1 row with samples as columns
            clinical_df = clinical_df.T
            clinical_df.columns = [trait]
            print(f"Transposed clinical data to shape: {clinical_df.shape}")
        
        # Now link clinical and genetic data
        linked_data = pd.concat([clinical_df, normalized_gene_data.T], axis=1)
        print(f"Linked data shape: {linked_data.shape}")
        
        # Check if we have at least one sample with trait value
        if trait in linked_data.columns:
            trait_count = linked_data[trait].count()
            print(f"Number of samples with trait values: {trait_count}")
            
            if trait_count > 0:
                # 3. Handle missing values systematically
                print("\nHandling missing values...")
                linked_data = handle_missing_values(linked_data, trait)
                print(f"After handling missing values, data shape: {linked_data.shape}")
                
                # Check if we still have samples after missing value handling
                if linked_data.shape[0] > 0:
                    # 4. Determine whether the trait and demographic features are biased
                    print("\nChecking for bias in features...")
                    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
                else:
                    print("Error: All samples were removed during missing value handling.")
                    is_biased = True
            else:
                print("No samples have valid trait values. Dataset cannot be used.")
                is_biased = True
        else:
            print(f"Trait column '{trait}' not found in linked data.")
            is_biased = True
    else:
        print("Cannot link data: clinical or genetic data is missing.")
        if not is_trait_available:
            print(f"Reason: Trait column '{trait}' is not available in clinical data.")
        if normalized_gene_data.empty:
            print("Reason: Gene expression data is empty.")
        linked_data = pd.DataFrame()
        is_biased = True
        
except Exception as e:
    print(f"Error in linking clinical and genetic data: {e}")
    linked_data = pd.DataFrame()
    is_biased = True

# 5. Final quality validation
print("\nPerforming final validation...")
note = ""
if 'linked_data' in locals() and linked_data.empty:
    if not normalized_gene_data.empty and not clinical_df.empty:
        note = "Dataset failed in linking phase. Clinical data structure could not be properly matched with gene expression data."
    elif normalized_gene_data.empty:
        note = "Dataset failed in gene mapping phase. Could not properly map probe IDs to gene symbols."
    elif clinical_df.empty:
        note = "Dataset failed in clinical data extraction. No valid trait information available."
elif 'is_biased' in locals() and is_biased:
    note = "Dataset passed initial processing but contains severely biased trait distribution."

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=not normalized_gene_data.empty,  # Set based on actual gene data availability
    is_trait_available=is_trait_available,
    is_biased=is_biased if 'is_biased' in locals() else True,
    df=linked_data if 'linked_data' in locals() and not linked_data.empty else pd.DataFrame(),
    note=note
)

# 6. Save linked data if usable
if is_usable and 'linked_data' in locals() and not linked_data.empty:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    
    # Save linked data
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Dataset not usable for {trait} association studies. Data not saved.")


Normalizing gene symbols...
After normalization: 24018 unique gene symbols


Normalized gene expression data saved to ../../output/preprocess/Kidney_Clear_Cell_Carcinoma/gene_data/GSE119958.csv

Checking trait availability from previous analysis...
Error loading clinical data: [Errno 2] No such file or directory: '../../output/preprocess/Kidney_Clear_Cell_Carcinoma/clinical_data/GSE119958.csv'

Linking clinical and genetic data...
Cannot link data: clinical or genetic data is missing.
Reason: Trait column 'Kidney_Clear_Cell_Carcinoma' is not available in clinical data.

Performing final validation...
Abnormality detected in the cohort: GSE119958. Preprocessing failed.
Dataset not usable for Kidney_Clear_Cell_Carcinoma association studies. Data not saved.
